In [3]:
#!/usr/bin/env python3
import torch
import cv2
import time
import os
import logging
import numpy as np
import pandas as pd
from ultralytics import YOLO
from typing import Dict, Tuple, Optional, List
from collections import deque
import re

# OCR imports
try:
    import pytesseract
    TESSERACT_AVAILABLE = True
except ImportError:
    TESSERACT_AVAILABLE = False

try:
    import easyocr
    EASYOCR_AVAILABLE = True
except ImportError:
    EASYOCR_AVAILABLE = False

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

class GPUManager:
    """GPU memory management"""
    
    @staticmethod
    def setup():
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.backends.cudnn.benchmark = True
            torch.backends.cudnn.deterministic = False
            
            gpu_count = torch.cuda.device_count()
            current_device = torch.cuda.current_device()
            gpu_name = torch.cuda.get_device_name(current_device)
            memory_total = torch.cuda.get_device_properties(current_device).total_memory / 1024**3

            logger.info(f"CUDA Available: {gpu_count} GPU(s)")
            logger.info(f"Current GPU: {gpu_name}")
            logger.info(f"GPU Memory: {memory_total:.1f} GB")
            return True
        logger.warning("Using CPU (slower)")
        return False
    
    @staticmethod
    def monitor():
        if torch.cuda.is_available():
            device = torch.cuda.current_device()
            used = torch.cuda.memory_allocated(device) / 1024**3
            total = torch.cuda.get_device_properties(device).total_memory / 1024**3
            utilization = (used / total) * 100
            
            if utilization > 85:
                torch.cuda.empty_cache()
            return utilization
        return 0.0

class FPSCounter:
    """Simple FPS calculator"""
    def __init__(self, buffer_size=30):
        self.timestamps = deque(maxlen=buffer_size)
    
    def tick(self):
        self.timestamps.append(time.time())
    
    def get_fps(self) -> float:
        if len(self.timestamps) < 2:
            return 0.0
        return len(self.timestamps) / (self.timestamps[-1] - self.timestamps[0])

class ChileanPlateValidator:
    """Chilean license plate format validation and correction"""
    
    ALLOWED_LETTERS = set("BCDFGHJKLPRSTWVXYZ")
    MODERN_PATTERN = re.compile(r'^[BCDFGHJKLPRSTWVXYZ]{4}\d{2}$')
    LEGACY_PATTERN = re.compile(r'^[A-Z]{2}\d{4}$')
    
    @staticmethod
    def clean_and_correct(text: str) -> str:
        """Clean and correct OCR text for Chilean plates"""
        if not text:
            return ""
        
        # Basic cleaning
        cleaned = ''.join(c.upper() for c in text if c.isalnum())
        if len(cleaned) != 6:
            return cleaned
        
        # Apply corrections based on expected format
        result = ""
        expected_format = "LLLLNN" if cleaned[:4].isalpha() else "LLNNNN"
        
        for i, char in enumerate(cleaned):
            if i >= len(expected_format):
                break
            
            if expected_format[i] == 'L':  # Should be letter
                corrected = ChileanPlateValidator._correct_to_letter(char)
            else:  # Should be number
                corrected = ChileanPlateValidator._correct_to_number(char)
            
            result += corrected
        
        return result
    
    @staticmethod
    def _correct_to_letter(char: str) -> str:
        """Convert character to valid Chilean plate letter"""
        # Number to letter mapping
        if char.isdigit():
            mapping = {'0': 'D', '1': 'L', '2': 'Z', '3': 'B', '4': 'H', 
                      '5': 'S', '6': 'G', '7': 'T', '8': 'B', '9': 'G'}
            char = mapping.get(char, char)
        
        # Handle forbidden letters
        if char not in ChileanPlateValidator.ALLOWED_LETTERS:
            forbidden_map = {
                'A': 'B', 'E': 'F', 'I': 'L', 'O': 'D', 'U': 'V',
                'M': 'W', 'N': 'W', 'Ñ': 'W', 'Q': 'G'
            }
            char = forbidden_map.get(char, 'B')  # Safe fallback
        
        return char
    
    @staticmethod
    def _correct_to_number(char: str) -> str:
        """Convert character to number"""
        if char.isdigit():
            return char
        
        letter_to_number = {
            'O': '0', 'I': '1', 'L': '1', 'S': '5', 'B': '8', 
            'G': '6', 'Z': '2', 'T': '7', 'D': '0', 'F': '7'
        }
        return letter_to_number.get(char, '0')
    
    @staticmethod
    def is_valid_format(text: str) -> str:
        """Check format and return type"""
        if not text or len(text) != 6:
            return 'invalid'
        
        if ChileanPlateValidator.MODERN_PATTERN.match(text):
            return 'modern'
        elif ChileanPlateValidator.LEGACY_PATTERN.match(text):
            return 'legacy'
        else:
            return 'invalid'

class OCREngine:
    """Simplified OCR processing"""
    
    def __init__(self, engine="easyocr"):
        self.engine = engine
        self.reader = None
        
        if engine == "easyocr" and EASYOCR_AVAILABLE:
            self.reader = easyocr.Reader(['en'], gpu=torch.cuda.is_available(), verbose=False)
        elif engine == "tesseract" and TESSERACT_AVAILABLE:
            pass  # Use pytesseract directly
        else:
            raise RuntimeError(f"OCR engine {engine} not available")
    
    def extract_text(self, image: np.ndarray, use_validation: bool = True) -> Tuple[str, str]:
        """Extract text from plate image"""
        try:
            processed = self._preprocess_image(image)
            
            # Get raw OCR text
            if self.engine == "easyocr" and self.reader:
                results = self.reader.readtext(processed, detail=0, paragraph=False)
                raw_text = ''.join(results) if results else ""
            else:
                config = '--psm 7 --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
                raw_text = pytesseract.image_to_string(processed, config=config).strip()
            
            # Clean raw text
            clean_raw = raw_text.upper().replace(' ', '').replace('-', '')
            
            # Apply Chilean corrections if validation enabled
            if use_validation:
                corrected_text = ChileanPlateValidator.clean_and_correct(clean_raw)
                return raw_text, corrected_text
            else:
                return raw_text, clean_raw
                
        except Exception as e:
            logger.debug(f"OCR failed: {e}")
            return "", ""
    
    def _preprocess_image(self, image: np.ndarray) -> np.ndarray:
        """Preprocess image for OCR"""
        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image.copy()
        
        # Resize if too small
        h, w = gray.shape
        if h < 80 or w < 150:
            scale = max(100/h, 200/w)
            new_h, new_w = int(h * scale), int(w * scale)
            gray = cv2.resize(gray, (new_w, new_h), interpolation=cv2.INTER_CUBIC)
        
        # Enhance image
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(2,2))
        gray = clahe.apply(gray)
        gray = cv2.bilateralFilter(gray, 9, 75, 75)
        gray = cv2.convertScaleAbs(gray, alpha=1.2, beta=10)
        
        return gray

class PlateCandidate:
    """Single plate candidate with confidence tracking"""
    
    def __init__(self, text: str, detection_conf: float):
        self.text = text
        self.count = 1
        self.max_detection_conf = detection_conf
        self.format_type = ChileanPlateValidator.is_valid_format(text)
    
    def update(self, detection_conf: float):
        """Update candidate with new detection"""
        self.count += 1
        self.max_detection_conf = max(self.max_detection_conf, detection_conf)
    
    def get_confidence(self, total_detections: int, min_detections: int) -> float:
        """Calculate final confidence score"""
        # Base confidence from detection rate
        detection_rate = self.count / max(total_detections, 1)
        
        # Format bonus
        format_bonus = 0.3 if self.format_type != 'invalid' else 0.0
        
        # Stability bonus (consistent detections)
        stability_bonus = 0.2 if self.count >= min_detections else 0.0
        
        # Detection quality
        detection_quality = self.max_detection_conf
        
        # Combine all factors
        confidence = (detection_rate * 0.4 + 
                     format_bonus + 
                     stability_bonus + 
                     detection_quality * 0.1)
        
        return min(1.0, confidence)

class VehicleTracker:
    """Track single vehicle and its plate candidates"""
    
    def __init__(self, track_id: int, bbox: Tuple[int, int, int, int]):
        self.track_id = track_id
        self.bbox = bbox
        self.candidates = {}  # text -> PlateCandidate
        self.total_detections = 0
        self.last_seen_frame = 0
        self.last_ocr_frame = 0
        self.best_plate = ""
        self.best_confidence = 0.0
        self.raw_text = ""
    
    def update_detection(self, bbox: Tuple[int, int, int, int], frame_count: int,
                        raw_text: str = "", plate_text: str = "", detection_conf: float = 0.0):
        """Update vehicle with new detection"""
        self.bbox = bbox
        self.last_seen_frame = frame_count
        
        if plate_text and len(plate_text) >= 4:
            self.raw_text = raw_text
            self.total_detections += 1
            
            if plate_text in self.candidates:
                self.candidates[plate_text].update(detection_conf)
            else:
                self.candidates[plate_text] = PlateCandidate(plate_text, detection_conf)
            
            self._update_best_plate()
    
    def _update_best_plate(self, min_detections: int = 2):
        """Update best plate based on confidence"""
        if not self.candidates:
            return
        
        best_candidate = None
        best_conf = 0.0
        
        for candidate in self.candidates.values():
            conf = candidate.get_confidence(self.total_detections, min_detections)
            if conf > best_conf:
                best_conf = conf
                best_candidate = candidate
        
        if best_candidate:
            self.best_plate = best_candidate.text
            self.best_confidence = best_conf
    
    def should_run_ocr(self, frame_count: int, ocr_interval: int, ocr_validation: bool) -> bool:
        """Check if should run OCR on this frame"""
        frames_since_ocr = frame_count - self.last_ocr_frame
        interval = ocr_interval if ocr_validation else ocr_interval * 2
        
        if frames_since_ocr >= interval:
            self.last_ocr_frame = frame_count
            return True
        return False
    
    def get_plate_info(self) -> Dict:
        """Get current plate information"""
        format_type = ChileanPlateValidator.is_valid_format(self.best_plate) if self.best_plate else 'invalid'
        # Get OCR confidence from best candidate
        ocr_confidence = 0.0
        if self.best_plate and self.best_plate in self.candidates:
            ocr_confidence = self.candidates[self.best_plate].max_detection_conf
        
        return {
            'plate': self.best_plate,
            'raw': self.raw_text,
            'confidence': self.best_confidence,
            'ocr_confidence': ocr_confidence,
            'format_type': format_type,
            'valid': format_type != 'invalid',
            'detections': self.total_detections
        }

class PlateTracker:
    """Manage all vehicle tracking"""
    
    def __init__(self, min_detections=2, ocr_interval=5):
        self.vehicles = {}  # track_id -> VehicleTracker
        self.frame_count = 0
        self.min_detections = min_detections
        self.ocr_interval = ocr_interval
    
    def update_frame(self):
        self.frame_count += 1
    
    def update_vehicle(self, track_id: int, bbox: Tuple[int, int, int, int],
                      raw_text: str = "", plate_text: str = "", detection_conf: float = 0.0):
        """Update vehicle tracking"""
        if track_id not in self.vehicles:
            self.vehicles[track_id] = VehicleTracker(track_id, bbox)
        
        self.vehicles[track_id].update_detection(bbox, self.frame_count, raw_text, plate_text, detection_conf)
    
    def should_run_ocr(self, track_id: int, ocr_validation: bool) -> bool:
        """Check if should run OCR for vehicle"""
        if track_id not in self.vehicles:
            return True
        return self.vehicles[track_id].should_run_ocr(self.frame_count, self.ocr_interval, ocr_validation)
    
    def get_plate_info(self, track_id: int) -> Dict:
        """Get plate info for vehicle"""
        if track_id not in self.vehicles:
            return {'plate': '', 'raw': '', 'confidence': 0.0, 'ocr_confidence': 0.0, 'format_type': 'invalid', 'valid': False, 'detections': 0}
        return self.vehicles[track_id].get_plate_info()
    
    def get_all_detections(self) -> Dict:
        """Get all vehicles with plates"""
        return {track_id: vehicle.get_plate_info() 
                for track_id, vehicle in self.vehicles.items() 
                if vehicle.best_plate}

class SimpleDetector:
    """Simplified detector base class"""
    
    def __init__(self, model_path: str, confidence: float, device: str = 'auto'):
        self.device = 'cuda' if (device == 'auto' and torch.cuda.is_available()) else device
        self.model = YOLO(model_path)
        self.confidence = confidence
        
        if self.device == 'cuda':
            self.model.to('cuda')
        
        self._warmup()
    
    def _warmup(self):
        """Warm up model with properly formatted dummy input"""
        if self.device == 'cuda':
            # Create properly normalized dummy input (0-1 range)
            dummy = torch.randn(1, 3, 640, 640).cuda().abs()  # Ensure positive values
            dummy = dummy / dummy.max()  # Normalize to 0-1
            with torch.no_grad():
                for _ in range(3):
                    self.model(dummy)
            logger.info("Model warmup complete")

class VehicleDetector(SimpleDetector):
    """Vehicle detection and tracking"""

    def __init__(self, model_path: str, confidence: float = 0.5, device: str = 'auto',
                 tracker_config: str = "bytetrack.yaml"):
        super().__init__(model_path, confidence, device)
        self.tracker_config = self._setup_tracker_config(tracker_config)
        self.vehicle_classes = [2, 3, 5, 7]  # car, motorcycle, bus, truck

    def _setup_tracker_config(self, config_name: str) -> str:
        """Setup tracker configuration with proper path handling"""
        if config_name == "bytetrack.yaml":
            return config_name  # Use default
        
        # Check if custom config exists
        if os.path.exists(config_name):
            logger.info(f"Using custom tracker config: {config_name}")
            return config_name
        else:
            logger.warning(f"Custom tracker config not found: {config_name}, using default bytetrack.yaml")
            return "bytetrack.yaml"
                    
    def detect(self, frame: np.ndarray):
        """Detect and track vehicles"""
        try:           
            results = self.model.track(
                frame,
                classes=self.vehicle_classes,
                conf=self.confidence,
                device=self.device,
                tracker=self.tracker_config,
                persist=True,
                verbose=False,
                half=self.device == 'cuda',
                agnostic_nms=True,
                max_det=30,
                imgsz=640
            )
            return results[0] if results else None
        except Exception as e:
            logger.error(f"Vehicle detection failed: {e}")
            return None

class PlateDetector(SimpleDetector):
    """License plate detection"""
    
    def __init__(self, model_path: str, confidence: float = 0.25, device: str = 'auto'):
        super().__init__(model_path, confidence, device)
        self.min_size = (30, 15)  # min width, height
    
    def detect(self, vehicle_crop: np.ndarray) -> List[Tuple[np.ndarray, float]]:
        """Find plates in vehicle crop"""
        try:
            results = self.model(vehicle_crop,
                                conf=self.confidence, 
                                device=self.device,
                                verbose=False,
                                half=self.device == 'cuda',
                                agnostic_nms=True,
                                max_det=20,
                                imgsz=640
                            )
            plates = []
            
            if results and results[0].boxes is not None:
                for box in results[0].boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                    conf = box.conf[0].cpu().item()
                    
                    # Check minimum size
                    if (x2 - x1) >= self.min_size[0] and (y2 - y1) >= self.min_size[1]:
                        plate_crop = vehicle_crop[y1:y2, x1:x2]
                        if self._is_good_quality(plate_crop):
                            plates.append((plate_crop, conf))
            
            return sorted(plates, key=lambda x: x[1], reverse=True)
        except Exception as e:
            logger.debug(f"Plate detection failed: {e}")
            return []
    
    def _is_good_quality(self, plate_crop: np.ndarray) -> bool:
        """Check plate quality"""
        if plate_crop.size == 0:
            return False
        
        gray = cv2.cvtColor(plate_crop, cv2.COLOR_BGR2GRAY) if len(plate_crop.shape) == 3 else plate_crop
        return np.std(gray) > 15  # Basic contrast check

class DebugManager:
    """Centralized debug functionality"""
    
    def __init__(self, enabled: bool = False, output_dir: str = "debug"):
        self.enabled = enabled
        self.output_dir = output_dir
        if self.enabled:
            os.makedirs(self.output_dir, exist_ok=True)
    
    def save_plate_image(self, plate_img: np.ndarray, track_id: int, frame_count: int, 
                        raw_text: str, clean_text: str):
        """Save plate image with OCR results"""
        if not self.enabled:
            return
        
        filename = f"plate_{track_id}_frame_{frame_count}.jpg"
        filepath = os.path.join(self.output_dir, filename)
        cv2.imwrite(filepath, plate_img)
        
        # Save text info
        info_file = filepath.replace('.jpg', '_info.txt')
        with open(info_file, 'w', encoding='utf-8') as f:
            f.write(f"Track ID: {track_id}\n")
            f.write(f"Frame: {frame_count}\n")
            f.write(f"Raw OCR: {raw_text}\n")
            f.write(f"Clean Text: {clean_text}\n")
        
        logger.debug(f"Debug saved: {filename} - OCR: '{raw_text}' -> '{clean_text}'")
    
    def save_frame(self, frame: np.ndarray, frame_count: int):
        """Save current frame"""
        if not self.enabled:
            return
        
        filename = f"frame_{frame_count}.jpg"
        filepath = os.path.join(self.output_dir, filename)
        cv2.imwrite(filepath, frame)
        logger.info(f"Frame saved: {filename}")

class DisplayRenderer:
    """Handle display rendering based on mode"""
    
    def __init__(self, ocr_validation: bool = True):
        self.ocr_validation = ocr_validation
        self.class_names = {2: "Car", 3: "Motorcycle", 5: "Bus", 7: "Truck"}
        self.format_colors = {
            'modern': (60, 200, 60),    # Green
            'legacy': (200, 200, 60),   # Yellow  
            'invalid': (60, 60, 200),   # Red
        }
    
    def draw_annotations(self, frame: np.ndarray, detections: Dict, fps: float, total_plates: int) -> np.ndarray:
        """Draw all annotations on frame"""
        annotated = frame.copy()
        
        for track_id, info in detections.items():
            self._draw_vehicle_detection(annotated, track_id, info)
        
        self._draw_status_info(annotated, fps, total_plates, detections)
        return annotated
    
    def _draw_vehicle_detection(self, img: np.ndarray, track_id: int, info: Dict):
        """Draw single vehicle detection"""
        x1, y1, x2, y2 = info['bbox']
        vehicle_class = self.class_names.get(info.get('vehicle_class', 2), "Vehicle")
        
        # Get display color and confidence
        color = self._get_color(info)
        confidence = info['confidence']
        
        # Draw vehicle box
        thickness = max(1, int(confidence * 3))
        cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness)
        
        # Create and draw label
        label = self._create_label(vehicle_class, track_id, info)
        self._draw_label(img, (x1, y1), label, color)
    
    def _get_color(self, info: Dict) -> Tuple[int, int, int]:
        """Get color based on detection quality"""
        if self.ocr_validation:
            if info['valid'] and info['confidence'] > 0.7:
                return self.format_colors.get(info['format_type'], (60, 200, 60))
            elif info['valid']:
                return (100, 150, 50)
            elif info['plate']:
                return (50, 140, 140)  # Yellow for detected but invalid
            else:
                return (120, 120, 255)  # Red for no detection
        else:
            # Raw mode - color by plate length
            plate_len = len(info['plate'])
            if plate_len >= 5:
                return (60, 200, 60)    # Green
            elif plate_len >= 3:
                return (50, 140, 140)   # Yellow
            else:
                return (120, 120, 255)  # Red
    
    def _create_label(self, vehicle_class: str, track_id: int, info: Dict) -> str:
        """Create display label based on mode"""
        base_label = f"{vehicle_class}:{track_id}"
        
        if not info['plate']:
            return base_label
        
        if self.ocr_validation:
            # Validation mode: show plate, format, validity, confidence
            format_type = info['format_type'].upper()[:3]
            status = "OK" if info['valid'] else "NO"
            confidence = info['confidence']
            return f"{base_label} {info['plate']} {format_type} {status} ({confidence:.2f})"
        else:
            # Raw mode: show OCR output, length, and OCR confidence
            plate = info['plate']
            length = len(plate)
            ocr_conf = info['ocr_confidence']
            return f"{base_label} {plate} L:{length} ({ocr_conf:.2f})"
    
    def _draw_label(self, img: np.ndarray, pos: Tuple[int, int], label: str, color: Tuple[int, int, int]):
        """Draw label with background"""
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.5
        thickness = 1
        
        (text_w, text_h), _ = cv2.getTextSize(label, font, font_scale, thickness)
        x, y = pos
        
        # Background rectangle
        cv2.rectangle(img, (x, y-text_h-10), (x+text_w+6, y), color, -1)
        cv2.rectangle(img, (x, y-text_h-10), (x+text_w+6, y), (255, 255, 255), 1)
        
        # Text
        cv2.putText(img, label, (x+3, y-5), font, font_scale, (255, 255, 255), thickness)
    
    def _draw_status_info(self, img: np.ndarray, fps: float, total_plates: int, detections: Dict):
        """Draw system status information"""
        # Main status
        status = f"FPS: {fps:.1f} | Plates: {total_plates}"
        cv2.rectangle(img, (5, 5), (250, 25), (40, 40, 40), -1)
        cv2.putText(img, status, (8, 18), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
        
        # Mode indicator
        mode = "VALIDATION" if self.ocr_validation else "RAW"
        cv2.rectangle(img, (5, 30), (120, 50), (40, 40, 40), -1)
        cv2.putText(img, f"Mode: {mode}", (8, 43), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0), 1)
        
        # GPU usage
        if torch.cuda.is_available():
            gpu_usage = GPUManager.monitor()
            gpu_color = (0, 255, 0) if gpu_usage < 70 else (0, 255, 255) if gpu_usage < 85 else (0, 0, 255)
            cv2.rectangle(img, (5, 55), (100, 75), (40, 40, 40), -1)
            cv2.putText(img, f"GPU: {gpu_usage:.1f}%", (8, 68), cv2.FONT_HERSHEY_SIMPLEX, 0.4, gpu_color, 1)

class PlateDetectionSystem:
    """Main system orchestrator"""
    
    def __init__(self, vehicle_model: str, plate_model: str, ocr_engine: str = "easyocr", **kwargs):
        # Initialize components
        self.ocr_validation = kwargs.get('ocr_validation', True)
        self.vehicle_padding = kwargs.get('vehicle_padding', 10)
        
        # Core components
        self.vehicle_detector = VehicleDetector(
            vehicle_model,
            kwargs.get('vehicle_confidence', 0.5),
            kwargs.get('tracker_config', 'bytetrack.yaml'))
        
        self.plate_detector = PlateDetector(
            plate_model,
            kwargs.get('plate_confidence', 0.25))
        
        self.ocr = OCREngine(ocr_engine)
        
        self.tracker = PlateTracker(kwargs.get('min_detections', 2), kwargs.get('ocr_interval', 5))
        
        # Support components
        self.renderer = DisplayRenderer(self.ocr_validation)
        self.debug = DebugManager(kwargs.get('debug_mode', False))
        self.fps_counter = FPSCounter()
        
        logger.info(f"System initialized - Validation: {self.ocr_validation}")
    
    def process_frame(self, frame: np.ndarray) -> Dict:
        """Process single frame"""
        self.fps_counter.tick()
        self.tracker.update_frame()
        
        # Detect vehicles
        vehicle_results = self.vehicle_detector.detect(frame)
        if not vehicle_results or vehicle_results.boxes is None:
            return {}
        
        detections = {}
        
        # Process each vehicle
        for box in vehicle_results.boxes:
            if not hasattr(box, 'id') or box.id is None:
                continue
            
            track_id = int(box.id[0].cpu().item())
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
            vehicle_class = int(box.cls[0].cpu().item()) if hasattr(box, 'cls') else 2
            bbox = (x1, y1, x2, y2)
            
            # Update tracker
            self.tracker.update_vehicle(track_id, bbox)
            
            # Run OCR if needed
            if self.tracker.should_run_ocr(track_id, self.ocr_validation):
                self._process_vehicle_ocr(frame, track_id, bbox)
            
            # Get plate info for display
            plate_info = self.tracker.get_plate_info(track_id)
            detections[track_id] = {
                'bbox': bbox,
                'vehicle_class': vehicle_class,
                **plate_info
            }
        
        return detections
    
    def _process_vehicle_ocr(self, frame: np.ndarray, track_id: int, bbox: Tuple[int, int, int, int]):
        """Process OCR for single vehicle"""
        # Extract vehicle crop
        x1, y1, x2, y2 = bbox
        pad = self.vehicle_padding
        
        crop_x1 = max(0, x1 - pad)
        crop_y1 = max(0, y1 - pad)
        crop_x2 = min(frame.shape[1], x2 + pad)
        crop_y2 = min(frame.shape[0], y2 + pad)
        
        vehicle_crop = frame[crop_y1:crop_y2, crop_x1:crop_x2]
        
        # Detect plates in vehicle
        plates = self.plate_detector.detect(vehicle_crop)
        
        # Process best plate
        for plate_img, detection_conf in plates[:1]:  # Only process best plate
            raw_text, clean_text = self.ocr.extract_text(plate_img, self.ocr_validation)
            
            # Debug save
            self.debug.save_plate_image(plate_img, track_id, self.tracker.frame_count, raw_text, clean_text)
            
            # Update tracker with results
            if clean_text and len(clean_text) >= 3:
                self.tracker.update_vehicle(track_id, bbox, raw_text, clean_text, detection_conf)
    
    def draw_annotations(self, frame: np.ndarray, detections: Dict) -> np.ndarray:
        """Draw annotations on frame"""
        fps = self.fps_counter.get_fps()
        total_plates = len(self.tracker.get_all_detections())
        return self.renderer.draw_annotations(frame, detections, fps, total_plates)
    
    def save_results(self, csv_path: str):
        """Save detection results to CSV"""
        detections = self.tracker.get_all_detections()
        
        if not detections:
            logger.info("No plates detected to save")
            return
        
        results = []
        for track_id, info in detections.items():
            vehicle = self.tracker.vehicles[track_id]
            
            result = {
                'track_id': track_id,
                'plate_text': info['plate'],
                'raw_text': info['raw'],
                'confidence': info['confidence'],
                'valid_format': info['valid'],
                'format_type': info['format_type'],
                'detections': info['detections'],
                'bbox_x1': vehicle.bbox[0],
                'bbox_y1': vehicle.bbox[1],
                'bbox_x2': vehicle.bbox[2],
                'bbox_y2': vehicle.bbox[3]
            }
            results.append(result)
        
        # Sort by confidence
        results.sort(key=lambda x: x['confidence'], reverse=True)
        
        # Save to CSV
        df = pd.DataFrame(results)
        os.makedirs(os.path.dirname(csv_path), exist_ok=True)
        df.to_csv(csv_path, index=False)
        
        # Log statistics
        valid_count = sum(1 for r in results if r['valid_format'])
        logger.info(f"Saved {len(results)} detections ({valid_count} valid) to {csv_path}")

class VideoProcessor:
    """Process video files with progress monitoring"""
    
    def __init__(self, system: PlateDetectionSystem, **kwargs):
        self.system = system
        self.frame_skip = kwargs.get('frame_skip', 1)
        self.show_display = kwargs.get('show_display', True)
        self.debug = kwargs.get('debug_mode', False)
    
    def process_video(self, input_path: str, output_path: str, results_path: str):
        """Process entire video"""
        # Validate input
        if not os.path.exists(input_path):
            raise FileNotFoundError(f"Input video not found: {input_path}")
        
        # Setup video capture and writer
        cap = cv2.VideoCapture(input_path)
        cap.set(cv2.CAP_PROP_BUFFERSIZE, 10)
        
        if not cap.isOpened():
            raise ValueError(f"Cannot open video: {input_path}")
        
        video_info = self._get_video_info(cap)
        writer = self._setup_writer(output_path, video_info)
        
        try:
            self._process_loop(cap, writer, video_info)
        except KeyboardInterrupt:
            logger.info("Processing interrupted by user")
        finally:
            self._cleanup(cap, writer, results_path)
    
    def _get_video_info(self, cap) -> Dict:
        """Get video properties"""
        return {
            'fps': int(cap.get(cv2.CAP_PROP_FPS)),
            'width': int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            'height': int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
            'total_frames': int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        }
    
    def _setup_writer(self, output_path: str, video_info: Dict):
        """Setup video writer"""
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        return cv2.VideoWriter(output_path, fourcc, video_info['fps'], 
                             (video_info['width'], video_info['height']))
    
    def _process_loop(self, cap, writer, video_info: Dict):
        """Main processing loop"""
        frame_count = 0
        start_time = time.time()
        last_processed_frame = None
        
        logger.info(f"Processing: {video_info['width']}x{video_info['height']} @ {video_info['fps']}fps")
        logger.info(f"Total frames: {video_info['total_frames']}")
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_count += 1
            
            # Process frame based on skip interval
            if frame_count % (self.frame_skip + 1) == 0:
                detections = self.system.process_frame(frame)
                processed_frame = self.system.draw_annotations(frame, detections)
                last_processed_frame = processed_frame
            elif last_processed_frame is not None:
                processed_frame = last_processed_frame
            else:
                processed_frame = frame
            
            # Write frame
            writer.write(processed_frame)
            
            # Handle display and user input
            if self.show_display:
                self._handle_display(processed_frame, frame_count)
            
            # Progress logging
            if frame_count % 150 == 0:
                self._log_progress(frame_count, video_info['total_frames'], start_time)
    
    def _handle_display(self, frame: np.ndarray, frame_count: int):
        """Handle display window and keyboard input"""
        cv2.imshow('Chilean License Plate Detection', frame)
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q'):
            raise KeyboardInterrupt
        elif key == ord('s'):
            self.system.debug.save_frame(frame, frame_count)
        elif key == ord(' '):  # Pause
            logger.info("Paused - press any key to continue")
            cv2.waitKey(0)
    
    def _log_progress(self, frame_count: int, total_frames: int, start_time: float):
        """Log processing progress"""
        elapsed = time.time() - start_time
        fps = frame_count / elapsed if elapsed > 0 else 0
        plates = len(self.system.tracker.get_all_detections())
        progress = (frame_count / total_frames) * 100
        
        # ETA calculation
        eta_seconds = (total_frames - frame_count) / fps if fps > 0 else 0
        eta_min, eta_sec = divmod(int(eta_seconds), 60)
        
        logger.info(f"Progress: {frame_count}/{total_frames} ({progress:.1f}%) | "
                   f"FPS: {fps:.1f} | Plates: {plates} | ETA: {eta_min}m{eta_sec:02d}s")
        
        # GPU monitoring
        if torch.cuda.is_available():
            gpu_usage = GPUManager.monitor()
            if gpu_usage > 80:
                logger.warning(f"High GPU usage: {gpu_usage:.1f}%")
    
    def _cleanup(self, cap, writer, results_path: str):
        """Cleanup resources and save results"""
        cap.release()
        writer.release()
        cv2.destroyAllWindows()
        
        # Save results
        self.system.save_results(results_path)
        
        # Final statistics
        total_vehicles = len(self.system.tracker.vehicles)
        total_plates = len(self.system.tracker.get_all_detections())
        valid_plates = sum(1 for info in self.system.tracker.get_all_detections().values() 
                          if info['valid'])
        
        logger.info("=" * 50)
        logger.info("PROCESSING COMPLETED")
        logger.info(f"Vehicles tracked: {total_vehicles}")
        logger.info(f"Plates detected: {total_plates}")
        logger.info(f"Valid plates: {valid_plates}")
        logger.info(f"Results saved: {results_path}")
        logger.info("=" * 50)

def process_video(vehicle_model: str, plate_model: str, input_video: str,
                 output_video: str, results_csv: str, **kwargs):
    """Main video processing function"""
    
    # Validate files exist
    for path in [vehicle_model, plate_model, input_video]:
        if not os.path.exists(path):
            raise FileNotFoundError(f"File not found: {path}")
    
    # Initialize system
    system = PlateDetectionSystem(vehicle_model, plate_model, **kwargs)
    processor = VideoProcessor(system, **kwargs)
    
    # Process video
    processor.process_video(input_video, output_video, results_csv)

def main():
    """Main function with optimized configuration"""
    
    # Initialize GPU
    cuda_available = GPUManager.setup()
    
    # Configuration
    config = {
        # Model paths
        'vehicle_model': "../checkpoints/model_weights/yolov8s.pt",
        'plate_model': "../checkpoints/model_weights/license_plate_detector.pt", 
        
        # Input/Output
        'input_video': "../data/input/Video4.mp4",
        'output_video': "../data/output/tracked_plates.mp4",
        'results_csv': "../data/output/plate_results.csv",
        
        # Processing settings
        'frame_skip': 2,
        'ocr_engine': "easyocr",
        'ocr_validation': True,  # Set to False for raw mode
        'show_display': True,
        'debug_mode': False,
        'tracker_config': 'custom_bytetrack.yaml',
        
        # Detection thresholds
        'vehicle_confidence': 0.5,
        'plate_confidence': 0.1,
        
        # Tracking settings
        'min_detections': 2,
        'ocr_interval': 3,
        'vehicle_padding': 10,
        
        # Device
        'device': 'auto'
    }
    
    print("=" * 70)
    print("CHILEAN LICENSE PLATE DETECTION SYSTEM")
    print("=" * 70)
    
    mode = "VALIDATION MODE" if config['ocr_validation'] else "RAW MODE"
    print(f"Current Mode: {mode}")
    
    if config['ocr_validation']:
        print("• Validates Chilean format (modern LLLLNN, legacy LLNNNN)")
        print("Allowed letters: B,C,D,F,G,H,J,K,L,P,R,S,T,V,W,X,Y,Z")
        print("Excluded: M,N,Ñ,Q (similar to 0/O) and vowels A,E,I,O,U")
        print("• Shows format type and validity")
    else:
        print("• Shows raw OCR output only")
        print("• No format validation")
    
    print(f"Debug Mode: {config['debug_mode']}")
    print("-" * 70)
    
    try:
        process_video(
            config['vehicle_model'],
            config['plate_model'], 
            config['input_video'],
            config['output_video'],
            config['results_csv'],
            **{k: v for k, v in config.items() if k not in 
               ['vehicle_model', 'plate_model', 'input_video', 'output_video', 'results_csv']}
        )
        print("\nProcessing completed successfully!")
        
    except Exception as e:
        print(f"\nError: {e}")
        logger.error(f"Processing failed: {e}")

if __name__ == "__main__":
    main()

2025-08-30 21:17:52,888 - CUDA Available: 1 GPU(s)
2025-08-30 21:17:52,889 - Current GPU: NVIDIA GeForce GTX 960M
2025-08-30 21:17:52,890 - GPU Memory: 1.9 GB


CHILEAN LICENSE PLATE DETECTION SYSTEM
Current Mode: VALIDATION MODE
• Validates Chilean format (modern LLLLNN, legacy LLNNNN)
Allowed letters: B,C,D,F,G,H,J,K,L,P,R,S,T,V,W,X,Y,Z
Excluded: M,N,Ñ,Q (similar to 0/O) and vowels A,E,I,O,U
• Shows format type and validity
Debug Mode: False
----------------------------------------------------------------------

0: 640x640 (no detections), 23.2ms
Speed: 0.4ms preprocess, 23.2ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 23.3ms
Speed: 0.0ms preprocess, 23.3ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 23.5ms
Speed: 0.0ms preprocess, 23.5ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 640)


2025-08-30 21:17:53,281 - Model warmup complete
2025-08-30 21:17:56,155 - System initialized - Validation: True
2025-08-30 21:17:56,169 - Processing: 854x480 @ 29fps
2025-08-30 21:17:56,169 - Total frames: 9646
2025-08-30 21:18:05,736 - Progress: 150/9646 (1.6%) | FPS: 15.7 | Plates: 4 | ETA: 10m05s
2025-08-30 21:18:13,418 - Progress: 300/9646 (3.1%) | FPS: 17.4 | Plates: 6 | ETA: 8m57s
2025-08-30 21:18:21,177 - Progress: 450/9646 (4.7%) | FPS: 18.0 | Plates: 7 | ETA: 8m31s
2025-08-30 21:18:29,730 - Progress: 600/9646 (6.2%) | FPS: 17.9 | Plates: 7 | ETA: 8m25s
2025-08-30 21:18:35,724 - Processing interrupted by user
2025-08-30 21:18:35,732 - Saved 8 detections (6 valid) to ../data/output/plate_results.csv
2025-08-30 21:18:35,733 - ==================================================
2025-08-30 21:18:35,734 - PROCESSING COMPLETED
2025-08-30 21:18:35,735 - Vehicles tracked: 36
2025-08-30 21:18:35,736 - Plates detected: 8
2025-08-30 21:18:35,738 - Valid plates: 6
2025-08-30 21:18:35,739 - 


Processing completed successfully!
